<a href="https://colab.research.google.com/github/lendy0/mcu112/blob/main/Final_exam_0618.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DB-API與底層資料庫互動示例：

①程式碼 ⇆ ②使用DB-API ⇆ ③資料庫驅動程式 ⇆ ④底層資料庫(如SQLite、MySQL等)

- connect
- cursor
- execute
- fetchall/fetchone

舉例來說，SQLite 是用C寫的一種體積很小的關聯式資料庫，它的資料庫就是一個檔案，所以經常被整合到應用程式中，在 iOS 和 Android 的 App 中都可以看到。Python 內置了 SQLite3 的驅動程式，所以，在 Python 中使用 SQLite 是不需要再安裝任何東西，就可以直接導入( import )使用，直接操作 SQLite 資料庫。

要操作關聯式資料庫，首先需要建立資料庫連線，一個資料庫的連線稱為一個 Connection。透過資料庫的連線，可以建立游標( Cursor )，通過 Cursor 執行 SQL 語句，然後得到執行結果。

建立資料庫
```python
import sqlite3 as sqlite

# connect to SQLite
con = sqlite.connect('db_web.db')

# Create a Connection
cur = con.cursor()

# Drop users table if already exsist.
cur.execute("DROP TABLE IF EXISTS users")

# Create users table  in db_web database
sql ='''CREATE TABLE "users" (
    "UID"    INTEGER PRIMARY KEY AUTOINCREMENT,
    "UNAME"    TEXT,
    "CONTACT"    TEXT
)'''
cur.execute(sql)

# commit changes
con.commit()

# close the connection
con.close()
```

# 查全部

**設定 con.row_factory=sqlite.Row**
```python
def index():
    con=sqlite.connect("db_web.db")
    # 設定 con.row_factory=sqlite.Row
    con.row_factory=sqlite.Row
    cur=con.cursor()
    cur.execute("select * from users")
    data=cur.fetchall()
    print(data)
    for row in data:
        # 可以使用欄位名稱取值
        print(row['UID'],row['UNAME'],row['CONTACT'])
```



**設定 con.row_factory=sqlite.Row** 結果

```text
[<sqlite3.Row object at 0x7c5a1375b5e0>]
1 lendy 1234567
```

**未設定 con.row_factory=sqlite.Row 差異**

```python
def index1():
    con=sqlite.connect("db_web.db")
    cur=con.cursor()
    cur.execute("select * from users")
    data=cur.fetchall()
    # 取出來是list中包含多筆tuple
    print(data)
    for (uid,uname,contact) in data:
        print(uid,uname,contact)
index1()
```
結果

```text
[(1, 'lendy', '1234567')]
1 lendy 1234567
```


# 新增
```python
def add_user():
    uname='lendy'
    contact='1234567'
    con=sqlite.connect("db_web.db")
    cur=con.cursor()
    cur.execute("insert into users(UNAME,CONTACT) values (?,?)",(uname,contact))
    con.commit()
```
        



# 修改
```python
def edit_user(uid):
    uname='lendy1'
    contact='12345671'
    con=sqlite.connect("db_web.db")
    cur=con.cursor()
    cur.execute("update users set UNAME=?,CONTACT=? where UID=?",(uname,contact,uid))
    con.commit()
```




# 刪除
```python
def delete_user(uid):
    con=sqlite.connect("db_web.db")
    cur=con.cursor()
    cur.execute("delete from users where UID=?",(uid,))
    con.commit()

```

# 第一題、請依照下面順序輸出並截圖
- 查全部(index())
- 新增(add_user())
- 查全部(index())
- 修改(edit_user(1))
- 查全部(index())
- 刪除(delete_user(1))
- 查全部(index())

# 第二題、安裝 ngrok 與 pyngrok

**step 1、下載 ngrok client**

```text
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
```

**step 2、解壓送下載的 ngrok client**
```text
!tar -xvf /content/ngrok-v3-stable-linux-amd64.tgz
```

**step 3、置入 ngrok 的 AuthToken**

Get your AuthToken from ngrok using this link:
```text
https://dashboard.ngrok.com/get-started/your-authtoken
```
在 colab 上的設定方法。
```text
!./ngrok config add-authtoken 2hisD7vbbDdF2md0zu1LsAp6ScI_6RbgQaSfn2XnRvdxoLxEA
```

**step 4、安裝 pyngrok**

```text
!pip install pyngrok
```

**step 5、讓 Flask 可以有一個外部 URL**

- 使用 pyngrok
- 取得 外部 URL
  - ngrok.connect(port).public_url

**範例程式**
```python
# import Flask from flask module
from flask import Flask

# import pyngrok to run the app using ngrok
from pyngrok import ngrok

#app name
app = Flask(__name__)

port = "5000"

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\" ")

@app.route("/")
def hello():
    return "<Title>打上你的學號</Title><h1>hello 請打上你的姓名</h1>"

if __name__ == "__main__":
    app.run(port=port)

```

# 第三題、結合 Flask、Jinja 2、DB 做 CRUD

**step 1、放入 HTML 檔案**
- /templates
  - /layout.html
  - /index.html
  - /add_user.html
  - /edit_user.html

**step2、執行下面範例程式**

```python
# import Flask from flask module
from flask import Flask,render_template,request,redirect,url_for,flash

from pyngrok import ngrok
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\" ")

# import sqlite3
import sqlite3 as sqlite

app = Flask(__name__) #app name

@app.route("/")
@app.route("/index")
def index():
    con=sqlite.connect("db_web.db")
    con.row_factory=sqlite.Row
    cur=con.cursor()
    cur.execute("select * from users")
    data=cur.fetchall()

    return render_template("index.html",datas=data)

@app.route("/add_user",methods=['POST','GET'])
def add_user():
    if request.method=='POST':
        uname=request.form['uname']
        contact=request.form['contact']
        con=sqlite.connect("db_web.db")
        cur=con.cursor()
        cur.execute("insert into users(UNAME,CONTACT) values (?,?)",(uname,contact))
        con.commit()
        flash('User Added','success')
        return redirect(url_for("index"))
    return render_template("add_user.html")

@app.route("/edit_user/<string:uid>",methods=['POST','GET'])
def edit_user(uid):
    if request.method=='POST':
        uname=request.form['uname']
        contact=request.form['contact']
        con=sqlite.connect("db_web.db")
        cur=con.cursor()
        cur.execute("update users set UNAME=?,CONTACT=? where UID=?",(uname,contact,uid))
        con.commit()
        flash('User Updated','success')
        return redirect(url_for("index"))
    con=sqlite.connect("db_web.db")
    con.row_factory=sqlite.Row
    cur=con.cursor()
    cur.execute("select * from users where UID=?",(uid,))
    data=cur.fetchone()
    return render_template("edit_user.html",datas=data)

@app.route("/delete_user/<string:uid>",methods=['GET'])
def delete_user(uid):
    con=sqlite.connect("db_web.db")
    cur=con.cursor()
    cur.execute("delete from users where UID=?",(uid,))
    con.commit()
    flash('User Deleted','warning')
    return redirect(url_for("index"))

if __name__=='__main__':
    app.secret_key='admin123'
    app.run()
```